In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Trees_Exercise").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 20:07:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/14 20:07:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


24/03/14 20:07:50 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
data = spark.read.csv("dog_food.csv", inferSchema=True, header=True)
data.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



In [4]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [5]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [6]:
assembler = VectorAssembler(inputCols=["A", "B", "C", "D"], outputCol="features")

In [7]:
output = assembler.transform(data)

In [8]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [9]:
final_data = output.select("features", "Spoiled")

In [10]:
train_data, test_data = final_data.randomSplit([0.7, 0.3], seed=20)

In [11]:
rfc = RandomForestClassifier(labelCol="Spoiled", featuresCol="features")

In [12]:
rfc_model = rfc.fit(train_data)

In [18]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0268, 1: 0.0291, 2: 0.9178, 3: 0.0263})

In [13]:
rfc_preds = rfc_model.transform(test_data)

In [14]:
my_binary_eval = BinaryClassificationEvaluator(labelCol="Spoiled")

In [15]:
print("RFC")
print(my_binary_eval.evaluate(rfc_preds))

RFC
0.9892125134843581


In [16]:
acc_eval = MulticlassClassificationEvaluator(labelCol="Spoiled", metricName="accuracy")

In [17]:
rfc_acc = acc_eval.evaluate(rfc_preds)
rfc_acc

0.9784172661870504